In [1]:
import pyquil

In [60]:
from pyquil.quil import Program 
from pyquil.gates import *
from pyquil.api import QVMConnection

In [1]:
import numpy as np
import scipy
from scipy.linalg import expm, sinm, cosm

In [ ]:
import sys
sys.path.append("/Users/Daniel/Rigetti/pyquil") # CHANGE THIS TO YOUR ABSOLUTE PATH TO THE PYQUIL FOLDER

In [77]:
K = np.array([[0.5065, 0.2425], 
              [0.2425, 0.4935]])
gamma = 2
F = K + 1/gamma*np.eye(2)

[[ 1.0065  0.2425]
 [ 0.2425  0.9935]]


In [108]:
class Inverse():
    def __init__(self, F, program, qvm, theta0, theta1, theta2):
        self.F = F
        self.program = program
        self.qvm = qvm
        self.first, self.second, self.y, self.anc = (0,1,2,3)
        
        self.theta0 = theta0
        self.theta1 = theta1
        self.theta2 = theta2

#     def expmatrix(self, val,matrix):  
#         return expm(val*1j*self.F)
    
    def controlMake(self, M):
        """Creates control gates from M.
        param:
            M: (matrix) to control
        returns:
            (matrix) controlled."""
        zero = np.zeros((2,2))
        I = np.eye(2)
        top = np.concatenate((I, zero), axis = 1)
        bottom = np.concatenate((zero, M), axis = 1)
        res = np.concatenate((top, bottom), axis = 0)
        return res
    
    def CRY(self, angle):
        """Creates control RY gate.
        param:
            angle: (float) by how much to rotate
        returns:
            matrix."""
        Y = np.array([[0, -1j], [1j, 0]])
        expY = expm(-angle/2. * 1j * Y)
        return self.controlMake(expY)
    
    def CCRY(self, angle):
        Y = np.array([[0, -1j], [1j, 0]])
        expY = expm(-angle/2. * 1j * Y)
        
        ccry = np.eye(8)
        ccry[6][6] = expY[0][0]
        ccry[6][7] = expY[0][1]
        ccry[7][6] = expY[1][0]
        ccry[7][7] = expY[1][1]

        return ccry
    
    def NCCRY(self, angle):
        Y = np.array([[0, -1j], [1j, 0]])
        expY = expm(-angle/2. * 1j * Y)
        
        cncry = np.eye(8)
        cncry[2][2] = expY[0][0]
        cncry[2][3] = expY[0][1]
        cncry[3][2] = expY[1][0]
        cncry[3][3] = expY[1][1]
        
        return cncry
        
    def HGate(self):
        return self.controlMake(np.sqrt(0.5) * np.array([[1,1], [1,-1]]))
    
    
    def main(self):
        # add hadamards
        self.program += H(self.first)
        self.program += H(self.second)
        
        # add the exponent gates
        expF = expm(np.pi*1j*self.F)
        expF = self.controlMake(expF)
        self.program = self.program.defgate("expF", expF)
        self.program.inst(("expF", self.first, self.y))
        
        expFhalf = expm(np.pi/2.*1j*self.F)
        expFhalf = self.controlMake(expFhalf)
        self.program = self.program.defgate("expFhalf", expFhalf)
        self.program.inst(("expFhalf", self.second, self.y))
        
        self.program += SWAP(self.first, self.second)
        self.program += H(self.second)
        
        #S inverse
        self.program += CPHASE(-np.pi/2, self.second, self.first) # right order of qubits?
        
        self.program += H(self.first)
        
        CRYpi4 = self.CRY(np.pi/4.)
        self.program = self.program.defgate("CRYpi4", CRYpi4)
        self.program.inst(("CRYpi4", self.second, self.anc))
        
        CRYpi8 = self.CRY(np.pi/8.)
        self.program = self.program.defgate("CRYpi8", CRYpi8)
        self.program.inst(("CRYpi8", self.first, self.anc))  
        
        self.program += H(self.first)
        
        self.program += CPHASE(np.pi/2, self.second, self.first) # right order of qubits?
        
        self.program += H(self.second)
        
        self.program += SWAP(self.first, self.second)
        
        minusExpFhalf = expm(-np.pi/2.*1j*self.F)
        minusExpFhalf = self.controlMake(minusExpFhalf)
        self.program = self.program.defgate("minusExpFhalf", minusExpFhalf)
        self.program.inst(("minusExpFhalf", self.second, self.y))
        
        minusExpF = expm(-np.pi*1j*self.F)
        minusExpF = self.controlMake(minusExpF)
        self.program = self.program.defgate("minusExpF", minusExpF)
        self.program.inst(("minusExpF", self.second, self.y))
        
        self.program += H(self.first)
        self.program += H(self.second)
        
        theta1Gate = self.NCCRY(self.theta1)
        self.program = self.program.defgate("theta1Gate", theta1Gate)
        self.program.inst(("theta1Gate", self.anc, self.y, self.second))
        
        theta2Gate = self.CCRY(self.theta2)
        self.program = self.program.defgate("theta2Gate", theta2Gate)
        self.program.inst(("theta2Gate", self.anc, self.y, self.second))
        
        theta0Gate = self.CRY(-self.theta0)
        self.program = self.program.defgate("theta0Gate", theta0Gate)
        self.program.inst(("theta0Gate", self.anc, self.second))
        
        CH = self.HGate()
        self.program = self.program.defgate("CH", CH)
        self.program.inst(("CH", self.y, self.anc))
        
        self.program += SWAP(self.first, self.anc)
        
        self.program += MEASURE(self.first, [0])
        
        print(self.program)
        
        return self.qvm.run(self.program, classical_addresses=[0], trials=10)
        
        
        

        
        
        


        


In [109]:
inverse = Inverse(F, Program(), QVMConnection(), np.arctan(0.987/0.159), np.arctan(0.987/0.159), np.arctan( 0.354/0.935))
inverse.main()

DEFGATE expF:
    1.0, 0.0, 0.0, 0.0
    0.0, 1.0, 0.0, 0.0
    0.0, 0.0, -0.72338088163488157-0.018500241159870121i, -7.8816046023640395e-17-0.69020130481053732i
    0.0, 0.0, -7.998239811928397e-17-0.6902013048105371i, -0.72338088163488157+0.018500241159870281i

DEFGATE expFhalf:
    1.0, 0.0, 0.0, 0.0
    0.0, 1.0, 0.0, 0.0
    0.0, 0.0, -0.0099648727332346913+0.92827282671499167i, -0.37176640581683762+7.6867799821281707e-17i
    0.0, 0.0, -0.37176640581683762+7.4155369628925033e-17i, 0.0099648727332350279+0.92827282671499156i

DEFGATE CRYpi4:
    1.0, 0.0, 0.0, 0.0
    0.0, 1.0, 0.0, 0.0
    0.0, 0.0, 0.92387953251128674, -0.38268343236508978
    0.0, 0.0, 0.38268343236508978, 0.92387953251128663

DEFGATE CRYpi8:
    1.0, 0.0, 0.0, 0.0
    0.0, 1.0, 0.0, 0.0
    0.0, 0.0, 0.98078528040323043, -0.19509032201612828
    0.0, 0.0, 0.19509032201612828, 0.98078528040323043

DEFGATE minusExpFhalf:
    1.0, 0.0, 0.0, 0.0
    0.0, 1.0, 0.0, 0.0
    0.0, 0.0, -0.0099648727332346913-0.9282728

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:55: ComplexWarning: Casting complex values to real discards the imaginary part
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:56: ComplexWarning: Casting complex values to real discards the imaginary part
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:57: ComplexWarning: Casting complex values to real discards the imaginary part
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:58: ComplexWarning: Casting complex values to real discards the imaginary part
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:43: ComplexWarning: Casting complex values to real discards the imaginary part
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:44: ComplexWarning: Casting complex values to real discards the imaginary part
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:45: ComplexWarning: Casting complex values to real discards the imaginary part
/usr/local/lib/pytho

[[0], [0], [0], [0], [0], [0], [0], [1], [0], [0]]

In [ ]:
H(1)

In [98]:
np.arctan(0.935/0.354)

1.2088648147915153